# Introduction
I use this notebook multiple times across the life of project, both as a starting overview and a point of reference between various cleaning and transformation steps.

It uses Google Colab form entries and a personalized version of the [cookiecutter data science](https://drivendata.github.io/cookiecutter-data-science/) file structure to simplify file selection.

In addition to basic built-in `pandas` overview tools, like `.info()` and `.describe()`, I added my own functions for common tasks including:
*  Creating a singular table of the same data from `.info()` and `.describe()`
    * Writing the table to `.csv`
    * Writing the table to `.xlsx` with frozen column headers and a `Notes` column, which I use to take notes about processing steps
*  Displaying an overview of unique values by column

# Set Up

## Authorize Google Drive
Follow pop up prompts to authorize Drive access. May not work with non-Chrome browsers depending on ad block and privacy settings.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Library imports

In [ ]:
#general analysis
import pandas as pd
import pprint as ppr
import re
import numpy as np

#file management
from pathlib import Path
from datetime import datetime

#stop words counter
#from collections import Counter

## Display Preferences

In [ ]:
#current preferences
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) #change column display width
#pd.set_option('display.precision', 2) #displays 2 decimal places on all numbers
pd.set_option('display.float_format',  '{:.2f}'.format)
pd.set_option('display.memory_usage', 'deep')

# File Handling
This section uses parametized forms in Google Colab to simplify file selection.

It may require running the same cells multiple times depending on how much information is needed to select the intended file/directory.

In [ ]:
file_path = ''
project = ''

In [ ]:
#main project path
project_dir = Path.cwd().joinpath("drive", "MyDrive", project)
project_dir

PosixPath('/content/drive/MyDrive/data_analysis/dunnhumby')

In [ ]:
#input files path
input_dir = project_dir.joinpath(folders)
input_dir

PosixPath('/content/drive/MyDrive/data_analysis/dunnhumby/data/raw')

In [ ]:
#output files path
output_dir = project_dir.joinpath("notebooks", "eda")
output_dir

PosixPath('/content/drive/MyDrive/data_analysis/dunnhumby/notebooks/eda')

In [ ]:
#unique marker for new files
today = datetime.now().strftime("%m-%d-%Y_%H%M%S")

### Read into pandas dataframe

In [ ]:
df = pd.read_csv(file_path,
#                        usecols= cols,
#                      sep='\t',
#                        nrows=100,
#                       engine='python',
#                     encoding='ISO-8859-1'
                        )

# DataFrame Overview

## Row and Column Count

In [ ]:
df.shape

(277100, 22)

## `.info()`

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277100 entries, 0 to 277099
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 277100 non-null  int64  
 1   SHOP_DATE                 277100 non-null  int64  
 2   SHOP_WEEKDAY              277100 non-null  int64  
 3   SHOP_HOUR                 277100 non-null  int64  
 4   QUANTITY                  277100 non-null  int64  
 5   SPEND                     277100 non-null  float64
 6   PROD_CODE                 277100 non-null  object 
 7   PROD_CODE_10              277100 non-null  object 
 8   PROD_CODE_20              277100 non-null  object 
 9   PROD_CODE_30              277100 non-null  object 
 10  PROD_CODE_40              277100 non-null  object 
 11  CUST_CODE                 226383 non-null  object 
 12  CUST_PRICE_SENSITIVITY    226383 non-null  object 
 13  CUST_LIFESTAGE            198732 non-null  o

## Descriptive Stats

In [ ]:
df.describe(include='all')

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
count,277100.00,277100.00,277100.00,277100.00,277100.00,277100.00,277100,277100,277100,277100,277100,226383,226383,198732,277100.00,277100,277100,277100,277100,277100,277100,277100
unique,NaN,NaN,NaN,NaN,NaN,NaN,4368,242,87,30,9,18301,4,6,NaN,3,4,4,5,759,4,12
top,NaN,NaN,NaN,NaN,NaN,NaN,PRD0903052,CL00063,DEP00008,G00007,D00002,CUST0000413198,MM,OT,NaN,L,MM,Top Up,Fresh,STORE00696,LS,S02
freq,NaN,NaN,NaN,NaN,NaN,NaN,5992,12215,23384,62306,136426,81,103544,60657,NaN,198321,144730,121307,142006,1306,173282,27863
mean,200701.00,20070270.69,4.02,14.97,1.51,1.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994104700412106.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.00,37.28,2.00,3.65,2.06,2.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233667.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,200701.00,20070226.00,1.00,8.00,1.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994104700000003.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,200701.00,20070227.00,2.00,12.00,1.00,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994104700209805.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,200701.00,20070301.00,4.00,15.00,1.00,1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994104700411380.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,200701.00,20070303.00,6.00,18.00,1.00,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994104700615334.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Testing

## Data Types, Memory Usage, Nulls, Value Counts

In [ ]:
#pandas documention re memory usage: base-2 representation; i.e. 1KB = 1024 bytes

In [ ]:
def get_dataframe_info(df):
    """
Recreates column-wise info from df.info() as a DataFrame to allow for easier viewing from CSV
    """
    df_dtypes = pd.DataFrame(df.dtypes, columns=['Data Types'])

    df_memory_usage = df.memory_usage(index=False, deep=True).to_frame(name='Memory (Bytes)')
    df_memory_usage['Memory (MB)'] = df_memory_usage['Memory (Bytes)']/1024/1024

    df_percent_null = ((1 - df.count() / len(df)) * 100).to_frame(name='Percent Null')

    df_described = df.describe(include='all').T


    df_info = pd.concat([df_dtypes, df_memory_usage, df_percent_null, df_described], axis=1)

    # Reassign column names
    new_column_names =  {'count': 'Count Null',
                        'unique': 'Unique Counts',
                        'top': 'Top Value',
                        'freq': 'Frequency',
                        'mean': 'Mean',
                        'std': 'Standard Deviation',
                        'min': 'Minimum',
                        '25%': '25%',
                        '50%': '50%',
                        '75%': '75%',
                        'max': 'Maximum'}
    df_info = df_info.rename(columns=new_column_names).rename_axis('Column Names')

#    df_info = df_info.set_index('column_names')

    return df_info

In [ ]:
df_info = get_dataframe_info(df)

In [ ]:
df_info.style.set_sticky(axis='index')

,Data Types,Memory (Bytes),Memory (MB),Percent Null,Count Null,Unique Counts,Top Value,Frequency,Mean,Standard Deviation,Minimum,25%,50%,75%,Maximum
Column Names,,,,,,,,,,,,,,,
SHOP_WEEK,int64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,200701.000000,0.000000,200701.000000,200701.000000,200701.000000,200701.000000,200701.000000
SHOP_DATE,int64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,20070270.693446,37.279772,20070226.000000,20070227.000000,20070301.000000,20070303.000000,20070304.000000
SHOP_WEEKDAY,int64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,4.024320,2.000169,1.000000,2.000000,4.000000,6.000000,7.000000
SHOP_HOUR,int64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,14.967286,3.652395,8.000000,12.000000,15.000000,18.000000,21.000000
QUANTITY,int64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,1.506348,2.062051,1.000000,1.000000,1.000000,1.000000,692.000000
SPEND,float64,2216800,2.114105,0.000000,277100.000000,nan,nan,nan,1.839542,2.429634,0.000000,0.760000,1.220000,2.080000,240.240000
PROD_CODE,object,18565700,17.705631,0.000000,277100,4368,PRD0903052,5992,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_10,object,17734400,16.912842,0.000000,277100,242,CL00063,12215,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_20,object,18011500,17.177105,0.000000,277100,87,DEP00008,23384,nan,nan,nan,nan,nan,nan,nan


### Save overview output in `.csv` and `.xlsx` format

In [ ]:
#write to csv file
df_info.to_csv(output_dir / f"{file_name}_overview_{today:%b-%d-%Y}.csv", index=False)

In [ ]:
#add a `notes` column for use with in a spreadsheet
df_info.insert(loc=0,
               column='Notes',
               value = '')

In [ ]:
#write to excel file
df_info.to_excel(output_dir / f"{file_name}_overview_{today:%b-%d-%Y}.xlsx",
                 sheet_name=f'Overview {file_name}',
                 freeze_panes=(1,2)
                 )

## Correlation Table
For numeric columns only

In [ ]:
df.corr(numeric_only=True)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,BASKET_ID
SHOP_WEEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHOP_DATE,NaN,1.00,0.42,-0.00,-0.00,0.00,-0.01
SHOP_WEEKDAY,NaN,0.42,1.00,0.01,-0.00,-0.00,0.00
SHOP_HOUR,NaN,-0.00,0.01,1.00,-0.01,0.01,-0.07
QUANTITY,NaN,-0.00,-0.00,-0.01,1.00,0.21,-0.00
SPEND,NaN,0.00,-0.00,0.01,0.21,1.00,-0.00
BASKET_ID,NaN,-0.01,0.00,-0.07,-0.00,-0.00,1.00


## Head and Tail Rows
First and last 10 rows

In [ ]:
df.head(10)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200701,20070304,1,17,1,1.13,PRD0900013,CL00015,DEP00004,G00003,D00001,CUST0000361701,MM,YA,994104700394636,L,MM,Full Shop,Mixed,STORE00001,LS,E02
1,200701,20070301,5,19,1,1.10,PRD0900015,CL00015,DEP00004,G00003,D00001,CUST0000871730,UM,NaN,994104700728646,S,MM,Small Shop,Fresh,STORE00001,LS,E02
2,200701,20070303,7,12,1,1.00,PRD0900015,CL00015,DEP00004,G00003,D00001,CUST0000949903,MM,PE,994104700780122,M,UM,Top Up,Fresh,STORE00001,LS,E02
3,200701,20070303,7,15,3,4.68,PRD0900049,CL00160,DEP00054,G00016,D00003,CUST0000644893,LA,PE,994104700579780,L,LA,Top Up,Mixed,STORE00001,LS,E02
4,200701,20070302,6,14,1,1.04,PRD0900055,CL00230,DEP00081,G00027,D00008,CUST0000926111,UM,OT,994104700764453,L,UM,Top Up,Fresh,STORE00001,LS,E02
5,200701,20070304,1,14,1,1.60,PRD0900062,CL00175,DEP00059,G00017,D00004,CUST0000605487,LA,YF,994104700553719,M,MM,Top Up,Mixed,STORE00001,LS,E02
6,200701,20070301,5,21,1,2.36,PRD0900071,CL00086,DEP00024,G00007,D00002,CUST0000666576,MM,YA,994104700593739,L,MM,Full Shop,Mixed,STORE00001,LS,E02
7,200701,20070304,1,12,1,1.05,PRD0900077,CL00150,DEP00052,G00015,D00003,CUST0000710863,LA,YF,994104700622917,L,MM,Full Shop,Mixed,STORE00001,LS,E02
8,200701,20070304,1,12,1,1.05,PRD0900077,CL00150,DEP00052,G00015,D00003,CUST0000795333,MM,NaN,994104700678351,L,MM,Full Shop,Mixed,STORE00001,LS,E02
9,200701,20070304,1,12,3,3.72,PRD0900086,CL00067,DEP00019,G00007,D00002,CUST0000710863,LA,YF,994104700622917,L,MM,Full Shop,Mixed,STORE00001,LS,E02


In [ ]:
df.tail(10)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
277090,200701,20070302,6,18,3,0.45,PRD0903473,CL00014,DEP00004,G00003,D00001,CUST0000948225,MM,NaN,994104700779080,M,MM,Small Shop,Fresh,STORE00558,SS,N03
277091,200701,20070301,5,20,3,4.02,PRD0904593,CL00010,DEP00003,G00002,D00001,NaN,NaN,NaN,994104700052086,L,MM,Top Up,Fresh,STORE00558,SS,N03
277092,200701,20070303,7,15,1,9.24,PRD0904735,CL00235,DEP00083,G00028,D00008,CUST0000026510,MM,OT,994104700175791,L,MM,Top Up,Fresh,STORE00558,SS,N03
277093,200701,20070301,5,14,1,0.42,PRD0900436,CL00148,DEP00052,G00015,D00003,CUST0000927651,MM,OF,994104700765515,L,MM,Full Shop,Mixed,STORE00558,SS,N03
277094,200701,20070227,3,21,1,0.93,PRD0900580,CL00047,DEP00012,G00004,D00002,CUST0000872634,UM,YA,994104700729203,M,UM,Small Shop,Fresh,STORE00558,SS,N03
277095,200701,20070227,3,18,1,1.91,PRD0901045,CL00197,DEP00067,G00021,D00005,CUST0000880716,UM,OT,994104700734658,S,UM,Small Shop,Mixed,STORE00558,SS,N03
277096,200701,20070302,6,12,1,1.22,PRD0901711,CL00163,DEP00055,G00016,D00003,CUST0000980541,MM,OT,994104700800088,M,LA,Top Up,Fresh,STORE00558,SS,N03
277097,200701,20070301,5,13,1,1.08,PRD0902098,CL00002,DEP00001,G00001,D00001,CUST0000024585,MM,YF,994104700174614,S,MM,Small Shop,Fresh,STORE00558,SS,N03
277098,200701,20070226,2,14,1,1.08,PRD0902098,CL00002,DEP00001,G00001,D00001,CUST0000067777,MM,OT,994104700202723,M,MM,Top Up,Mixed,STORE00558,SS,N03
277099,200701,20070227,3,11,1,2.51,PRD0903360,CL00100,DEP00033,G00009,D00002,CUST0000024585,MM,YF,994104700174615,L,MM,Top Up,Mixed,STORE00558,SS,N03


## Duplicated Rows

In [ ]:
df[df.duplicated(keep=False)]

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION


## Null Columns
List of columns with all NaN values

In [ ]:
list(df.columns[df.isnull().all(axis=0)])

[]

# Columns

## List of Column Names

In [ ]:
list(df.columns)

['SHOP_WEEK',
 'SHOP_DATE',
 'SHOP_WEEKDAY',
 'SHOP_HOUR',
 'QUANTITY',
 'SPEND',
 'PROD_CODE',
 'PROD_CODE_10',
 'PROD_CODE_20',
 'PROD_CODE_30',
 'PROD_CODE_40',
 'CUST_CODE',
 'CUST_PRICE_SENSITIVITY',
 'CUST_LIFESTAGE',
 'BASKET_ID',
 'BASKET_SIZE',
 'BASKET_PRICE_SENSITIVITY',
 'BASKET_TYPE',
 'BASKET_DOMINANT_MISSION',
 'STORE_CODE',
 'STORE_FORMAT',
 'STORE_REGION']

## Column Overview
This function loops over each column to produce the following info:
* name
* count of unique values
* datatype
* string to copy into a new cell, for displaying all value counts for a column

For value counts under the cutoff:
* a transposed display frame of all unique values

For value counts over the cutoff:
* transposed display frames of the top and bottom 10 values and counts


In [ ]:
def column_overview(df, columns, cutoff=100):
    """
    Display column name, count of unique values, and an easy to read dataframe of individual unique values and their counts

    Parameters
    ----------
    df: dataframe
    columns: list
    cutoff: int, maximum unique value count to display
    Recommend string or object columns with unclear or low expected unique values.

    Returns
    -------
    Prints out one result for each column in the provided list.
    """
    for col in columns:
        print("\n")
        print("Column name: " + col)
        num_unique = str(df[col].nunique())
        print(f"Number of Unique Values: {num_unique}")
        col_datatype = str(df[col].dtype)
        print(f"Column Datatype: {col_datatype}")
        print("\n")

        if int(num_unique) < cutoff:
          print("use line below for vertical results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          display(pd.DataFrame(df[col].value_counts(dropna=False)).T)
          print(u'\u2500' * 80)
        else:
          print(f"more than {cutoff} results, showing Top 10 and Bottom 10")
          print("use line below for complete results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          print("\n")
          print(f"Top 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).head(10)).T)
          print(f"Bottom 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).tail(10)).T)
          print(u'\u2500' * 80)


In [ ]:
column_overview(df, df.columns, 200)



Column name: SHOP_WEEK
Number of Unique Values: 1
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEK'].value_counts(dropna=False))


,200701
SHOP_WEEK,277100


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_DATE
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_DATE'].value_counts(dropna=False))


,20070302,20070228,20070301,20070304,20070303,20070226,20070227
SHOP_DATE,40639,40625,40216,39825,39665,38145,37985


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_WEEKDAY
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEKDAY'].value_counts(dropna=False))


...

────────────────────────────────────────────────────────────────────────────────


Column name: STORE_FORMAT
Number of Unique Values: 4
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_FORMAT'].value_counts(dropna=False))


,LS,MS,XLS,SS
STORE_FORMAT,173282,59923,23635,20260


────────────────────────────────────────────────────────────────────────────────


Column name: STORE_REGION
Number of Unique Values: 12
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_REGION'].value_counts(dropna=False))


,S02,N01,W02,N03,S03,W01,N02,S01,E03,E01,W03,E02
STORE_REGION,27863,27356,26160,25992,22970,22827,22793,22358,21076,19709,19016,18980


────────────────────────────────────────────────────────────────────────────────
